# Keyword Spotting Model - Homespace Capstone

### by ReDay Zarra

## Importing Libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

## Data Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255, # Feature Scaling
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale=1./255)

batchSize = 16 # Experiment with this
size = 64

training_set = train_datagen.flow_from_directory(
    "data/train",
    target_size = (size, size),
    batch_size = batchSize,
    class_mode = "categorical",
    classes = ['Aberto', 'Background Noise', 'Silencio'] 
)

validation_set = test_datagen.flow_from_directory(
    "data/validation",
    target_size = (size, size),
    batch_size = batchSize,
    class_mode = "categorical",
    classes = ['Aberto', 'Background Noise', 'Silencio']
)

test_set = test_datagen.flow_from_directory(
    "data/test",
    target_size = (size, size),
    batch_size = batchSize,
    class_mode = "categorical",
    classes = ['Aberto', 'Background Noise', 'Silencio'], 
    shuffle=False # Set shuffle to False to keep the order of test images for evaluation
)

## Building a Custom CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Define a function to build a model with varying parameters
def build_model(num_filters, kernel_size, dropout_rate):
    model = Sequential([
        Conv2D(num_filters, kernel_size, activation='relu', input_shape=(size, size, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(3, activation='softmax')
    ])
    return model

# Define a function to compile, train, and evaluate a model
def train_and_evaluate_model(model, optimizer, learning_rate):
    model.compile(optimizer=optimizer(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        training_set,
        steps_per_epoch=len(training_set),
        epochs=epochs,
        validation_data=validation_set,
        validation_steps=len(validation_set)
    )

    test_loss, test_accuracy = model.evaluate(test_set, steps=len(test_set))
    return test_accuracy

epochs = 25  # Adjust this value based on the convergence of your model

# Iterate over different model architectures and hyperparameters
best_accuracy = 0
best_parameters = None
for num_filters in [16, 32, 64]:
    for kernel_size in [(3, 3), (5, 5)]:
        for dropout_rate in [0.25, 0.5, 0.75]:
            for optimizer in [tf.keras.optimizers.Adam, tf.keras.optimizers.RMSprop]:
                for learning_rate in [0.001, 0.01]:
                    model = build_model(num_filters, kernel_size, dropout_rate)
                    accuracy = train_and_evaluate_model(model, optimizer, learning_rate)
                    
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_parameters = (num_filters, kernel_size, dropout_rate, optimizer, learning_rate)

# Train the best model with the best parameters found above
best_num_filters, best_kernel_size, best_dropout_rate, best_optimizer, best_learning_rate = best_parameters
final_model = build_model(best_num_filters, best_kernel_size, best_dropout_rate)

# Compile the final model
final_model.compile(optimizer=best_optimizer(learning_rate=best_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the final model
final_history = final_model.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=epochs,
    validation_data=validation_set,
    validation_steps=len(validation_set)
)

# Evaluate the final model
final_test_loss, final_test_accuracy = final_model.evaluate(test_set, steps=len(test_set))

In [ ]:
# Save the final model
final_model.save('my_custom_cnn_model.h5')

## Using the VGG16 Model

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
# Image Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

batchSize = 16
size = 224

training_set = train_datagen.flow_from_directory(
    "data/train",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical"
)

validation_set = test_datagen.flow_from_directory(
    "data/validation",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical"
)

test_set = test_datagen.flow_from_directory(
    "data/test",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical",
    shuffle=False
)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(size, size, 3))
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
def build_pretrained_model(base_model, dropout_rate):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(3, activation='softmax')
    ])
    return model

In [ ]:
pretrained_model = build_pretrained_model(base_model, dropout_rate=0.5)
pretrained_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

pretrained_history = pretrained_model.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=epochs,
    validation_data=validation_set,
    validation_steps=len(validation_set)
)

pretrained_test_loss, pretrained_test_accuracy = pretrained_model.evaluate(test_set, steps=len(test_set))

## Using the EfficientNetB0 Model

In [1]:
pip install -U efficientnet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from efficientnet.keras import EfficientNetB0

In [8]:
# Image Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
)

test_datagen = ImageDataGenerator(rescale=1./255)

batchSize = 16
size = 224

training_set = train_datagen.flow_from_directory(
    "data/train",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical"
)

validation_set = test_datagen.flow_from_directory(
    "data/validation",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical"
)

test_set = test_datagen.flow_from_directory(
    "data/test",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical",
    shuffle=False
)

Found 168 images belonging to 3 classes.
Found 36 images belonging to 3 classes.
Found 36 images belonging to 3 classes.


In [9]:
# Load the pre-trained EfficientNetB0 model without the top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(size, size, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Create a new model that uses the pre-trained model as its base
def build_pretrained_model(base_model, dropout_rate, number_of_classes):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(number_of_classes, activation='softmax')
    ])
    return model

# Define the number of classes for your specific problem
number_of_classes = 3
dropout_rate = 0.5

# Train and evaluate the new model
pretrained_model = build_pretrained_model(base_model, dropout_rate, number_of_classes)
pretrained_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 30

pretrained_history = pretrained_model.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=epochs,
    validation_data=validation_set,
    validation_steps=len(validation_set)
)

pretrained_test_loss, pretrained_test_accuracy = pretrained_model.evaluate(test_set, steps=len(test_set))


Epoch 1/30
11/11 [==============================] - 10s 526ms/step - loss: 1.0183 - accuracy: 0.4940 - val_loss: 0.7533 - val_accuracy: 0.6389
Epoch 2/30
11/11 [==============================] - 4s 401ms/step - loss: 0.6545 - accuracy: 0.7083 - val_loss: 0.4998 - val_accuracy: 0.7778
Epoch 3/30
11/11 [==============================] - 5s 413ms/step - loss: 0.5417 - accuracy: 0.7381 - val_loss: 0.4822 - val_accuracy: 0.7778
Epoch 4/30
11/11 [==============================] - 5s 410ms/step - loss: 0.4881 - accuracy: 0.7798 - val_loss: 0.4209 - val_accuracy: 0.8056
Epoch 5/30
11/11 [==============================] - 4s 396ms/step - loss: 0.5059 - accuracy: 0.7857 - val_loss: 0.3929 - val_accuracy: 0.8056
Epoch 6/30
11/11 [==============================] - 4s 400ms/step - loss: 0.3906 - accuracy: 0.8214 - val_loss: 0.3650 - val_accuracy: 0.8056
Epoch 7/30
11/11 [==============================] - 5s 433ms/step - loss: 0.3944 - accuracy: 0.8393 - val_loss: 0.3508 - val_accuracy: 0.7778
Epoch

## Quantization

In [11]:
pip install --user tensorflow_model_optimization

Note: you may need to restart the kernel to use updated packages.


In [12]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# Quantize the model
q_aware_pretrained_model = quantize_model(pretrained_model)
q_aware_pretrained_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                                  loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the quantized model
q_aware_pretrained_history = q_aware_pretrained_model.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=epochs,
    validation_data=validation_set,
    validation_steps=len(validation_set)
)

# Evaluate the quantized model
q_aware_pretrained_test_loss, q_aware_pretrained_test_accuracy = q_aware_pretrained_model.evaluate(
    test_set, steps=len(test_set)
)

ValueError: Quantizing a tf.keras Model inside another tf.keras Model is not supported.

## Convert to TensorFlow Lite

In [ ]:
import tempfile
import os

# Create a temporary directory
tempdir = tempfile.mkdtemp()

# Save the quantized model to the temporary directory
quantized_model_file = os.path.join(tempdir, 'quantized_model.tflite')
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_pretrained_model)
tflite_quantized_model = converter.convert()

# Save the quantized model as a .tflite file
with open(quantized_model_file, 'wb') as f:
    f.write(tflite_quantized_model)

## Convert to .CC File

In [ ]:
sudo apt-get install xxd

In [ ]:
xxd -i quantized_model.tflite > quantized_model.cc